In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Point, Polygon

# GeoPandas - dataframes with geometry for GIS applications

### get some data - `read_file` is the ticket for GeoJSON, shapefiles, GDB, etc.

In [ ]:
parks = gp.read_file('data/Madison_Parks.geojson')

### this now looks like a Pandas DataFrame but there's a special column `geometry`

In [ ]:
parks.head()

### also some important metadata particularly the [CRS](https://en.wikipedia.org/wiki/Spatial_reference_system)

In [ ]:
parks.crs

> ## pro tip: You can have multiple geometry columns but only one is _active_ -- this is important later as we do operations on GeoDataFrames. The column labeled `geometry` is typically the active one but you [you can change it](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_geometry.html).

## So what's up with these geometries? They are represented as [`shapely`](https://shapely.readthedocs.io/en/stable/manual.html) objects so can be:
- ### polygon / multi-polygon
- ### point / multi-point
- ### line / multi-line

## we can access with pandas `loc` and `iloc` references

In [ ]:
parks.iloc[1].geometry.type

In [ ]:
parks.loc[parks.ShortName=='Olin'].geometry.type

### There are other cool shapely properties like `area`

In [ ]:
parks.loc[parks.ShortName=='Olin'].geometry.area

### ruh-roh - what's up with this CRS and tiny area number?

In [ ]:
parks.crs

## area units in lat/long don't make sense. Let's project to something in meters (but how?)

In [ ]:
parks.to_crs(3071, inplace=True)
parks.crs

In [ ]:
parks.loc[parks.ShortName=='Olin'].geometry.area

### there are loads of useful methods for `shapely` objects for relationships between geometries (intersection, distance, etc.) but we will skip these for now because GeoPandas facilitates these things for entire geodataframes! #sick

## So back to GeoDataFrames.....we can look at them spatially as well with `plot()`

In [ ]:
parks.plot()

## easily make a chloropleth map using a selected column as the color (and add a legend)

In [ ]:
parks.columns

In [ ]:
parks.sample(4)

In [ ]:
parks.plot(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6})

## also a very cool interactive plot options with a basemap

In [ ]:
parks.explore(column='Acreage', cmap='magma')

## we can read in another shapefile

In [ ]:
hoods = gp.read_file('data/Neighborhood_Associations.geojson')

In [ ]:
hoods.sample(5)

### and we can plot these on top of each other

In [ ]:
ax_hood = hoods.plot()
# now plot the other one but specify which axis to plot on (ax=ax_hood)
parks.plot(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6}, ax=ax_hood)

## WAT! Why so far apart?

In [ ]:
hoods.crs

## we need to reproject. Geopandas uses `to_crs()` for this purpose

In [ ]:
# we can reproject, and set hoods to park crs 
hoods.to_crs(parks.crs, inplace=True)

In [ ]:
ax_hood = hoods.plot()
# now plot the other one but specify which axis to plot on (ax=ax_hood)
parks.plot(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6}, ax=ax_hood)

## or similarly with the interactive maps

In [ ]:
m_hood = hoods.explore()
parks.explore(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6}, m=m_hood)

## we can make a new geodataframe using shapely properties of the geometry - how about centroids?

## TEST YOUR SKILLS #0
- make a new geodataframe of the parks
- add a columns with centroids for each park
- plot an interactive window with the park centroids and the neighborhoods
- hints: 
    - remember the shapely methods are available for each geometry object (e.g. `centroid()`) 
    - you can loop over the column in a couple different ways
    - you can define which columns contains the geometry of a geodataframe
    - you will likely have to define the CRS

# Operations on and among geodataframes...do I need to use a GIS program?

## Dissolve

In [ ]:
hoods.dissolve() # note it defaults to filling all the columns with the first value

In [ ]:
ax=hoods.dissolve().plot()
hoods.plot(facecolor=None, edgecolor='orange', ax=ax)

## Convex Hull

In [ ]:
ax = hoods.dissolve().convex_hull.plot()
hoods.plot(facecolor=None, edgecolor='orange', ax=ax)

## Bounding Box

In [ ]:
hoods.bounds # that's per each row

In [ ]:
tb = hoods.total_bounds # this gives overall bounds
tb

## We can make a polygon from these coordinates with `shapely`

In [ ]:
from shapely.geometry import box

In [ ]:
bbox = box(tb[0], tb[1], tb[2], tb[3])
# pro tip - when passing a bunch of ordered arguments, '*' will unpack them #nice
bbox = box(*hoods.total_bounds)

## to make a GeoDataFrame from scratch, the minimum you need is geometry, but a crs is important, and some data will populate more columns

In [ ]:

hoods_boundary = gp.GeoDataFrame(data={'thing':['bounding_box']},geometry=[bbox], crs=hoods.crs)
hoods_boundary

In [ ]:
ax = hoods_boundary.plot()
hoods.plot(facecolor=None, edgecolor='orange', ax=ax)

# How about some spatial joins?

## we can bring in information based on locational overlap. Let's look at just a couple neighborhoods (Marquette and Tenny-Lapham) on the Isthmus

In [ ]:
isthmus = hoods.loc[hoods['NEIGHB_NAME'].str.contains('Marquette') | 
                   hoods['NEIGHB_NAME'].str.contains('Tenney')]
isthmus

In [ ]:
isthmus.explore()

In [ ]:
isthmus.sjoin(parks).explore()

In [ ]:
parks.sjoin(isthmus).explore()

### so, it matters which direction you join from. The geometry is preserved from the dataframe "on the left"
### equivalently, you can be more explicit in calling `sjoin`

In [ ]:
gp.sjoin(left_df=parks, right_df=isthmus).explore()

In [ ]:
isthmus_parks = gp.sjoin(left_df=parks, right_df=isthmus)

## we are going to use this `isthmus_parks` geoDataFrame a little later, but we want to trim out some unneeded and distracting columns. We can use `.drop()` just like with a regular Pandas DataFrame

In [ ]:
isthmus_parks.columns

In [ ]:
isthmus_parks.drop(columns=[ 'index_right','OBJECTID_right', 'NA_ID', 'STATUS', 'CLASSIFICA', 'Web',
       'ShapeSTArea', 'ShapeSTLength'], inplace=True)

# Let's explore the various predicates with a small intersecting box

In [ ]:
bbox = box(570600, 290000, 573100, 291700)
bounds = gp.GeoDataFrame(geometry=[bbox],crs=parks.crs)
bounds.plot()

## See [documentation](https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates) for full set of options for predicates. We'll just check out a couple options: `intersects`, `contains`, `within`

# TEST YOUR SKILLS #1
Using the `bounds` geodataframe you just made, write a function to visualize predicate behaviors.
- your function should accept a left geodataframe, a right geodataframe, and a string for the predicate
- your function should plot:
    - the left geodataframe in (default) blue
    - the result of the spatial join operation in another color
    - the right geodataframe in another color with outline only
- then you should set the title of the plot to the string predicate value used
- the geodataframes to test with are `isthmus_parks` and `bounds`
- your function should `return` the joined geodataframe

- a couple hints:
    - in the `plot` method are a couple args called `facecolor` and `edgecolor` that will help plot the rectangle
    - there are other predicates to try out 

- _advanced options_: if that was easy, you can try a couple other things like:
    - explore joins with points and lines in addition to just polygons
    - change around the `bounds` polygon dimensions 
    - use `explore()` to make an interactive map

# Spatial joins are particularly useful with collections of points. A common case is to add a polygon attribute to points falling within each polygon. Let's check out a bigger point dataset with all the trees on streets in Madison

In [ ]:
trees = gp.read_file('data/Street_Trees.geojson', index_col=0)
trees.plot(column='SPECIES')

## let's put this into the same crs as neighborhoods and join the data together so we can have a neighborhood attribute on the trees geodataframe

In [ ]:
trees.to_crs(hoods.crs, inplace=True)

In [ ]:
hoods.columns

## NOTE: if we pass only some columns of the GeoDataFrame, only those columns will be included in the result, which is cool. _But_ - must include the active geometry column as well!

In [ ]:
trees_with_hoods = trees[['SPECIES','DIAMETER','geometry']].sjoin(hoods[['NEIGHB_NAME','geometry']])
trees_with_hoods

# now we can do a groupby, for example, to find things like the average or max diameter of trees in each neighborhood

In [ ]:
trees_with_hoods.groupby('NEIGHB_NAME')['DIAMETER'].max().plot(kind='bar')

# we can flip this back to the original neighborhoods GeoDataFrame to make a more useful spatial plot

In [ ]:
hood_trees = hoods.copy()
tree_summary = trees_with_hoods.groupby('NEIGHB_NAME')['DIAMETER'].max()
hood_trees.merge(tree_summary,
                left_on = 'NEIGHB_NAME', right_on='NEIGHB_NAME').explore(column="DIAMETER")

## As we've seen, spatial joins are powerful, but they really only gather data from multiple collections. What if we want to actually calculate the amount of overlap among shapes? Or create new shapes based on instersection or not intersection of shapes? [`overlay`](https://geopandas.org/en/stable/docs/user_guide/set_operations.html?highlight=overlay) does these things.

In [ ]:
len(parks_cent), len(hoods)

In [ ]:
hoods

In [ ]:
trees = gp.read_file('data/Street_Trees.geojson', index_col=0)
trees

In [ ]:
trees.plot(markersize=.2, column='SPECIES')

# a little aside on spatial joins

In [ ]:
# let's zoom in on a small area to look at
croprot.plot()
plt.xlim([550000,555000])
plt.ylim([400000, 405000])


In [ ]:
# we can cook up a polygon and yank out only the polygons within it


In [ ]:
subset = Polygon([(550000,400000),
                  (550000,405000), 
                  (555000,405000),
                  (555000,400000),
                 (550000,400000)])
subgdf = gp.GeoDataFrame({'outline':['p0'],
                         'geometry':[subset]},
                        crs=croprot.crs)

In [ ]:
subgdf.plot()

In [ ]:
croprot_sub = gp.overlay(croprot, subgdf)

In [ ]:
# we need to cook up a dataframe real quick once
pt_df = gp.GeoDataFrame({'ptname':['pt0','pt1'],
                        'geometry':[Point(554000,403000),
                                   Point(552000,403000)]},
                       crs=croprot.crs)

In [ ]:
pt_df

In [ ]:
# we can pick a point at which we want attributes
ax = croprot_sub.plot()

pt_df.plot(marker='*', facecolor='orange', markersize=50, ax=ax)

# sadly, labeling points is BRUTAL
for x,y,label in zip(pt_df.geometry.x, pt_df.geometry.y, pt_df.ptname):
    ax.annotate(label, xy=(x,y), xytext=(3,3), size=14, textcoords = "offset points")


In [ ]:
gp.sjoin(croprot_sub, pt_df, predicate='intersects')

In [ ]:
# Let's add a polygon
poly_df = gp.GeoDataFrame({'polname':['p0'],
                        'geometry':[Polygon([(554000,403000),
                                            (552000,403000),
                                            (553000,402000),
                                            (554000,402000),
                                            (554000,403000)])]},
                       crs=croprot.crs)


In [ ]:
ax = croprot_sub.plot()

poly_df.plot(ax=ax, facecolor='none', edgecolor='orange')

In [ ]:
# there are a few ways to spatially join. Let's check them out

In [ ]:
ax = croprot_sub.plot()
gp.sjoin(croprot_sub, poly_df, predicate='intersects').plot(ax=ax, facecolor='k')
poly_df.plot(ax=ax, facecolor='none', edgecolor='orange')
plt.title('instersects')

In [ ]:
ax = croprot_sub.plot()
gp.sjoin(croprot_sub, poly_df, predicate='contains').plot(ax=ax, facecolor='k')
poly_df.plot(ax=ax, facecolor='none', edgecolor='orange')
plt.title('contains')

In [ ]:
ax = croprot_sub.plot()
gp.sjoin(croprot_sub, poly_df, predicate='within').plot(ax=ax, facecolor='k')
poly_df.plot(ax=ax, facecolor='none', edgecolor='orange')
plt.title('within')

In [ ]:
ax = croprot_sub.plot()
gp.sjoin(poly_df, croprot_sub, predicate='contains').plot(ax=ax, facecolor='k')
poly_df.plot(ax=ax, facecolor='none', edgecolor='orange')
plt.title('contains from the opposite direction')

# now back to the main goal

In [ ]:
croprot.plot(column='rotation', legend=True, figsize=(10,10))


In [ ]:
# we will need bounding boxes for the inset models
pfl_bbox = gp.read_file('../plainfield-lakes-lgr/pst_setup/postproc/shps/pfl_lgr_parent_bbox.shp')
plsnt_bbox = gp.read_file('../pleasant-lake-lgr/pst_setup/postproc/shps/plsnt_lgr_parent_bbox.shp')


In [ ]:
# we will need to trim this to the model boundary
regional_bbox = gp.read_file('../gis/centralsands/model_domain_poly.shp').to_crs(pfl_bbox.crs)
regional_bbox.crs

In [ ]:
# let's intersect the main coverage with just the two insets

In [ ]:
gp.overlay(croprot,pfl_bbox, how='intersection')

In [ ]:
# 'doh! let's reproject

In [ ]:
croprot = croprot.to_crs(pfl_bbox.crs)

In [ ]:
# WTH? NoneType???
croprot

In [ ]:
croprot = croprot.dropna()
croprot

In [ ]:
# try again
croprot = croprot.to_crs(pfl_bbox.crs)

In [ ]:
croprot.crs

In [ ]:
pfl_LU = gp.overlay(croprot,pfl_bbox, how='intersection')

In [ ]:
#sweet! no warning about crs

In [ ]:
croprot = gp.overlay(croprot, regional_bbox, how='intersection')

In [ ]:
ax = croprot.plot(column='rotation', legend=True, figsize=(10,10))
pfl_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
plsnt_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
regional_bbox.plot(ax=ax, facecolor="none", edgecolor='black')

In [ ]:
ax = pfl_LU.plot(column='rotation', legend=True)
pfl_bbox.plot(ax=ax, facecolor="none", edgecolor='black')

In [ ]:
# let's bring in the lakes to plot as well
alllakes = gp.read_file('../gis/centralsands/all_lakes.shp')

In [ ]:
# make sure in the correct CRS
assert alllakes.crs == pfl_bbox.crs

In [ ]:
pfl_lakes = gp.overlay(alllakes, pfl_bbox, how='intersection')
plsnt_lakes = gp.overlay(alllakes, plsnt_bbox, how='intersection')


In [ ]:
ax = pfl_LU.plot(column='rotation', legend=True)
pfl_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
pfl_lakes.plot(ax=ax, facecolor="blue")

In [ ]:
plsnt_LU = gp.overlay(croprot,plsnt_bbox, how='intersection')

In [ ]:
ax = plsnt_LU.plot(column='rotation', legend=True)
plsnt_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
plsnt_lakes.plot(ax=ax, facecolor="blue")

In [ ]:
# let's get nutty with the legend 

In [ ]:
plsnt_LU.rotation.unique()

In [ ]:
# grab some colors form the interwebs
# https://htmlcolorcodes.com/color-names/
rotcolors = {'non-ag':'Black',
             'cash_grain':'DarkGreen',
             'dairy': 'Orange',
             'potato/veg': 'SlateGrey',
             'pasture/hay': 'Gold'
            }
rotnames ={'non-ag':'Non Agriculture',
             'cash_grain':'Cash Grain',
             'dairy': 'Dairy',
             'potato/veg': 'Potato/Vegetable',
             'pasture/hay': 'Pasture/Hay'
            }

##### read in the non-ag landuse 

In [ ]:
nonag_lu = gp.read_file('shapefiles/no_irrigation_LU/irr_parcel_reclass_20210212/irr_parcel_reclass_20210212.shp')
nonag_lu.crs

In [ ]:
nonag_lu = nonag_lu.to_crs(pfl_bbox.crs)
nonag_lu.crs

In [ ]:
# read in the code lookup for nonag
nonag_lookup = pd.read_csv('shapefiles/no_irrigation_LU/wiscland_xref.csv', index_col = 0)

In [ ]:
nonag_lookup.head()


In [ ]:
nonag_lookup.loc[2100]['label']

In [ ]:
# we can do normal pandas things
nonag_lu.wl_lev2_cd = [nonag_lookup.loc[i,'label'] for i in nonag_lu.wl_lev2_cd]

In [ ]:
nonag_lu.loc[nonag_lu.wl_lev2_cd == 'Crop Rotation', 'wl_lev2_cd'] = 'Nonirrigated Agriculture'

In [ ]:
# and change "Crop Rotation" label to "Nonirrigated Agriculture"
ax = nonag_lu.plot(column = 'wl_lev2_cd', legend=True, figsize=(10,10))
regional_bbox.plot(ax=ax, edgecolor="k", facecolor='none' )

In [ ]:
from Figures import ReportFigures

rf = ReportFigures()
rf.set_style()
default_aspect = 6 / 8.0 # h/w
tall_aspect = 7 / 8.0
singlecolumn_width = 21/6.0
doublecolumn_width = 42/6.0

singlecolumn_size = (singlecolumn_width, singlecolumn_width * default_aspect)
singlecolumn_sizeT = (singlecolumn_width, singlecolumn_width * tall_aspect)
doublecolumn_size = (doublecolumn_width, doublecolumn_width * default_aspect)
doublecolumn_sizeT = (doublecolumn_width, doublecolumn_width * tall_aspect)

# now plot the regional model land use with and without irrigated ag

In [ ]:
fig, ax = plt.subplots(figsize=doublecolumn_sizeT)
for ct, dat in croprot.groupby('rotation'):
    ccol = rotcolors[ct]
    dat.plot(color=ccol,
            ax=ax)
plsnt_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
ax.annotate(text='Pleasant Lake\nInset Model', xy=plsnt_bbox.geometry.values[0].centroid.coords[0],
           horizontalalignment = 'center')
pfl_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
ax.annotate(text='Plainfield Lakes\nInset Model', xy=pfl_bbox.geometry.values[0].centroid.coords[0],
           horizontalalignment = 'center')
alllakes.plot(ax=ax, facecolor="blue")
regional_bbox.plot(ax=ax, facecolor="none", edgecolor='GoldenRod')

# set up legend patches using the color and long-name dicts from above
leg_patches =[mpatches.Patch(color=c, label=rotnames[v]) for v,c in rotcolors.items()]
leg_patches.append(mpatches.Patch(color='blue', label='Lakes'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='black', label='Inset Model Boundary'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='GoldenRod', label='Regional Model Boundary'))

ax.legend(bbox_to_anchor=(1.35, .96), handles=leg_patches, title='EXPLANATION')
ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.title('Crop Rotations for Regional Model')
plt.tight_layout()
plt.savefig('fig_x_regional_crop_rotations_map.pdf', bbox_inches='tight')

In [ ]:
nonag_lu.wl_lev2_cd.unique()

In [ ]:

nonag_colors = {
    'Nonirrigated Agriculture': 'SlateGrey',
    'Forage Grassland': 'Gold',
    'Idle Grassland':'PaleGoldenRod',
    'Broad-leaved Deciduous Forest':'ForestGreen', 
    'Coniferous Forest': 'DarkGreen', 
    'Emergent/Wet Meadow': 'DeepSkyBlue',
    'Forested Wetland': 'SlateBlue'
}

In [ ]:
fig, ax = plt.subplots(figsize=doublecolumn_sizeT)
for ct, dat in nonag_lu.groupby('wl_lev2_cd'):
    ccol = nonag_colors[ct]
    dat.plot(color=ccol,
            ax=ax)
plsnt_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
ax.annotate(text='Pleasant Lake\nInset Model', xy=plsnt_bbox.geometry.values[0].centroid.coords[0],
           horizontalalignment = 'center')
pfl_bbox.plot(ax=ax, facecolor="none", edgecolor='black')
ax.annotate(text='Plainfield Lakes\nInset Model', xy=pfl_bbox.geometry.values[0].centroid.coords[0],
           horizontalalignment = 'center')
alllakes.plot(ax=ax, facecolor="blue")
regional_bbox.plot(ax=ax, facecolor="none", edgecolor='GoldenRod')

# set up legend patches using the color and long-name dicts from above
leg_patches =[mpatches.Patch(color=c, label=v) for v,c in nonag_colors.items()]
leg_patches.append(mpatches.Patch(color='blue', label='Lakes'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='black', label='Inset Model Boundary'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='GoldenRod', label='Regional Model Boundary'))

ax.legend(bbox_to_anchor=(1.38, .96), handles=leg_patches, title='EXPLANATION')
ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.title('Crop Rotations for Regional Model')
plt.tight_layout()
plt.savefig('fig_x_regional_nonag_landuse_map.pdf', bbox_inches='tight')

In [ ]:
# now for insets
pfl_nonag = gp.overlay(nonag_lu, pfl_bbox, how='intersection')

In [ ]:
fig, ax = plt.subplots(2,1, figsize=doublecolumn_sizeT)
#pfl ag
for ct, dat in pfl_LU.groupby('rotation'):
    ccol = rotcolors[ct]
    dat.plot(color=ccol,
            ax=ax[0])
pfl_bbox.plot(ax=ax[0], facecolor="none", edgecolor='black')
pfl_lakes.plot(ax=ax[0], facecolor="blue")


# set up legend patches using the color and long-name dicts from above
leg_patches =[mpatches.Patch(color=c, label=rotnames[v]) for v,c in rotcolors.items()]
leg_patches.append(mpatches.Patch(color='blue', label='Lakes'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='black', label='Model Boundary'))
ax[0].legend(bbox_to_anchor=(1.5, 1.0), handles=leg_patches, title='EXPLANATION')
ax[0].xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[0].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[0].set_xlabel('Easting (m)')
ax[0].set_ylabel('Northing (m)')
rf.title(ax[0],'Crop Rotations for Plainfield Lakes Inset Model',
                capitalize=False,
                 wrap=500,
                subplot_prefix='A')

# pfl no ag 
for ct, dat in pfl_nonag.groupby('wl_lev2_cd'):
    ccol = nonag_colors[ct]
    dat.plot(color=ccol,
            ax=ax[1])
pfl_bbox.plot(ax=ax[1], facecolor="none", edgecolor='black')
pfl_lakes.plot(ax=ax[1], facecolor="blue")


# set up legend patches using the color and long-name dicts from above
leg_patches =[mpatches.Patch(color=c, label=v) for v,c in nonag_colors.items()]
leg_patches.append(mpatches.Patch(color='blue', label='Lakes'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='black', label='Model Boundary'))
ax[1].legend(bbox_to_anchor=(1.7, 1.0), handles=leg_patches, title='EXPLANATION')
ax[1].xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[1].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[1].set_xlabel('Easting (m)')
ax[1].set_ylabel('Northing (m)')
rf.title(ax[1],'No Irrigated Agriculture Land Use for Plainfield Lakes Inset Model',
                capitalize=False,
                 wrap=500,
                subplot_prefix='B')
plt.tight_layout()
plt.savefig('fig_x_pfl_inset_crop_rotations_map.pdf', bbox_inches='tight')





In [ ]:
plsnt_nonag = gp.overlay(nonag_lu, plsnt_bbox, how='intersection')

In [ ]:
fig, ax = plt.subplots(2,1, figsize=doublecolumn_sizeT)
#pfl ag
for ct, dat in plsnt_LU.groupby('rotation'):
    ccol = rotcolors[ct]
    dat.plot(color=ccol,
            ax=ax[0])
plsnt_bbox.plot(ax=ax[0], facecolor="none", edgecolor='black')
plsnt_lakes.plot(ax=ax[0], facecolor="blue")


# set up legend patches using the color and long-name dicts from above
leg_patches =[mpatches.Patch(color=c, label=rotnames[v]) for v,c in rotcolors.items()]
leg_patches.append(mpatches.Patch(color='blue', label='Lakes'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='black', label='Model Boundary'))
ax[0].legend(bbox_to_anchor=(1.6, 1.0), handles=leg_patches, title='EXPLANATION')
ax[0].xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[0].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[0].set_xlabel('Easting (m)')
ax[0].set_ylabel('Northing (m)')
rf.title(ax[0],'Crop Rotations for Pleasant Lake Inset Model',
                capitalize=False,
                 wrap=500,
                subplot_prefix='A')

# pfl no ag 
for ct, dat in plsnt_nonag.groupby('wl_lev2_cd'):
    ccol = nonag_colors[ct]
    dat.plot(color=ccol,
            ax=ax[1])
plsnt_bbox.plot(ax=ax[1], facecolor="none", edgecolor='black')
plsnt_lakes.plot(ax=ax[1], facecolor="blue")


# set up legend patches using the color and long-name dicts from above
leg_patches =[mpatches.Patch(color=c, label=v) for v,c in nonag_colors.items()]
leg_patches.append(mpatches.Patch(color='blue', label='Lakes'))
leg_patches.append(mpatches.Patch(facecolor='none',edgecolor='black', label='Model Boundary'))
ax[1].legend(bbox_to_anchor=(1.8, 1.0), handles=leg_patches, title='EXPLANATION')
ax[1].xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[1].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ax[1].set_xlabel('Easting (m)')
ax[1].set_ylabel('Northing (m)')
rf.title(ax[1],'No Irrigated Agriculture Land Use for Pleasant Lake Inset Model',
                capitalize=False,
                 wrap=500,
                subplot_prefix='B')
plt.tight_layout()
plt.savefig('fig_x_plsnt_inset_crop_rotations_map.pdf', bbox_inches='tight')